# Collision Avoidance - Data Collection（衝突回避 - データの収集)

basic motionのノートブックを実行し、JetBotを簡単に動かすことができ、お楽しみいただけたのではないでしょうか？ それはとてもいい経験です。でも、もっとすごいのは、JetBotは、自律的に動き回る事ができることです。

完全な自律走行はとてもハードなタスクで、この分野には多くの異なるアプローチが存在します。自律走行で直面する多くの問題は、より簡単で小さな問題に分割する事ができます。中でも最も重要で、解決すべき重要な問題は、ロボットが危険な状況に入るのを防ぐ事です。これを*collision avoidance*と呼びます。

この章では、非常に用途が広く、単体で動作するセンサーであるカメラと、ディープラーニングを用いてこの問題を解決することを経験できます。ニューラルネットワーク、カメラ、およびNVIDIA Jetson Nanoを使用して、JetBotに障害物を回避させる方法を学ぶことができます。

衝突を回避するためのアプローチは、JetBotの周りに仮想的な"safety bubble"を作り出す事でおこないます。"safety bubble"の中では、JetBotは、オブジェクトにぶつかることなく（または棚から落ちるなどのその他の危険な状況にならずに）円を描くように回転できます。

もちろん、JetBotはカメラに写るものしか認識することができないため、背後などの障害物を回避することはできません。しかし、JetBotがこれらの回避不能なシナリオに入るのを防ぐことはできます。

この方法は、実際やってみるととてもシンプルです。

最初に、"safety bubble"に違反する場所、つまり旋回したい場所にJetBotを手動で移動します。そして、``blocked``のラベルをつけます。ラベルとともに、JetBotが見ている画像もsnapshotとして保存します。

次に、JetBotを直進できる場所に手動で移動します。そして、``free``のラベルをつけます。同様に、ラベルとともにsnapshotを保存します。　

この`01_data_collection_JP.ipynb`では、このようにして`free`と`blocked`のデータを集めます。ラベルと画像をたくさん用意できれば、表示される画像に基づいてJetBotの"safety bubble"が侵害されているかどうかを正確に予測できるようになります。

> 重要なメモ: Jetbotの旋回は、ロボット筐体の中心ではなく、2つのWheelの中心で回転します。これは、ロボットの"safety bubble"が侵害されているかどうかを推定する場合に、覚えておくべき重要事項です。正確である必要はないので、心配はしないでください。ぶつかりそうな場合は、より大きな"safety bubble"を想定してください。JetBotが狭い場所に入ってしまい、Uターン出来なくなるシナリオに入らないように注意します。

### カメラの初期化と表示をおこないます

JetBotのカメラクラスは、CSI MIPI cameraを有効にするために使います。衝突回避に使うニューラルネットワークのモデルでは、224x224ピクセルの画像データを入力として使います。画像サイズが大きくなると、ニューラルネットワークモデルの学習や実行に必要とするメモリ量と処理時間が大幅に増えます。Jetson Nanoでは最適な値としてこのサイズを設定します。

カメラ映像を確認するためのウィジェットを作成して表示します。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import traitlets  # カメラ画像などのデータが更新されたときに、連動して処理を実行させるためにtraitletsライブラリを利用します。
import ipywidgets.widgets as widgets  # Jupyter標準のウィジェットを利用します。
from IPython.display import display  # ウィジェットを表示するためのdisplayライブラリを利用します。
from jetbot import Camera, bgr8_to_jpeg  # JetBot用に用意したカメラと画像変換ライブラリを利用します。

########################################
# カメラを有効化します。
# 画像はwidthとheightで指定したピクセルサイズにリサイズされます。
########################################
camera = Camera.instance(width=224, height=224)

########################################
# 画像表示用のウィジェットを用意します。
# widthとheightは表示するウィジェットの幅と高さです。
# カメラ画像サイズと一致する必要はありません。
########################################
image = widgets.Image(format='jpeg', width=224, height=224)

########################################
# traitletsライブラリを利用してカメラ画像データが更新されたときに、
# bgr8フォーマットをjpegフォーマットに変換してから
# 画像表示ウィジェットに反映するように設定します。
########################################
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

########################################
# 画像表示ウィジェットをブラウザに表示します。
########################################
display(image)

### 次は、データを保存するためのディレクトリを作成しましょう
データセットは「dataset」ディレクトリ以下に保存することにします。衝突回避では直進する場合と旋回する場合の2つのパターンが存在するため、    
直進と判断する時の画像を保存するための「dataset/free」ディレクトリと、    
旋回と判断する時の画像を保存するための「dataset/blocked」ディレクトリを作成します。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import os # ディレクトリ作成のためにpython標準のosライブラリを読み込みます。

########################################
# 作成するディレクトリパスを定義します。
########################################
blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

########################################
# データセット保存用のディレクトリを作成します。
# ディレクトリがすでに存在する場合、ディレクトリ作成関数がエラーを返す可能性があるため、
# ここは「try/except」ステートメントで囲みます。
########################################
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('ディレクトリが存在しているため、作成をスキップします。')

> Jupyterの左側のファイルブラウザをリフレッシュすれば、「dataset」ディレクトリが新規で生成された事がわかります。  
> 「File Browser(Ctrl + Shift + F)」の枠に表示されている「Reflesh File List」ボタンでリフレッシュできます。

### 画像データ保存ボタンの作成
次に、「free」と「blocked」に分けて保存するためのボタンと、それぞれ何枚保存されているかを表示するためのテキストボックスを用意して表示します。  
ボタンイベントはまだ作成していないので、ボタンを押しても保存されません。

In [ ]:
########################################
# ボタンウィジェットのサイズを定義します。
########################################
button_layout = widgets.Layout(width='128px', height='64px')

########################################
# ボタンウィジェットを作成します。
########################################
free_button = widgets.Button(description='freeに保存', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='blockedに保存', button_style='danger', layout=button_layout)

########################################
# 保存済みの画像ファイル数を表示するテキストボックスを作成します。
########################################
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

########################################
# 作成したウィジェットを表示します。
########################################
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

### 保存ボタンにイベントを実装
このボタンウィジェットは、descriptionとして「freeに保存」や「blockedに保存」と書いてはありますが、クリックされた時に何を実行するのかまだ定義されていません。  
そこでボタンの``on_clock``イベントにそれぞれのディレクトリにイメージを保存する関数を対応づけます。  
`image = widgets.Image(format='jpeg', width=224, height=224)`で定義された`image`変数は`image.value`にJPEG形式の画像データを持っているので、それをjpgファイルとして保存します。

(異なるマシン間でも) 同じファイル名にならないうようにPythonの``uuid``パッケージを使います。このユニークな識別子は、現在時刻とマシンアドレスなどの情報から生成されます。

> （注意！）このセルを複数回実行すると、イベントが複数個登録されてしまうため、ボタン1回のクリックで複数回保存処理が実行されてしまいます。  
> そのような場合は、カメラを停止させた後、Jupyter Kernelを停止させて、さらにdatasetディレクトリを削除してから、最初からやり直しとなります。

> （ファイル件数について）Jupyterによって「dataset/blocked/.ipynb_checkpoints」や「dataset/free/.ipynb_checkpoints」が作成されることがあります。このファイルは画像ファイルではありませんが、`os.listdir(blocked_dir)`によってファイルとして検出されるため、データ件数としてカウントされているように見えてしまいます。実際の画像ファイルの件数はこの分だけ異なる場合があります。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
from uuid import uuid1

########################################
# 画像ウィジェットが持つjpeg画像データを
# 指定されたディレクトリにjpgファイルとして保存します。
########################################
def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

########################################
# 「free」用に画像を保存します。
# また、「free」として保存されているファイル数の値を更新します。
########################################
def save_free():
    global free_dir, free_count  # free_dirとfree_countの定義とsave_free()関数の実行はともにメインスレッド上で行われるため、このglobal定義は省略可能です。
    save_snapshot(free_dir)  # save_snapshot()関数を呼び出して、「dataset/free」ディレクトリに現在imageウィジェットが持つjpeg画像データをjpgファイルとして保存します。
    free_count.value = len(os.listdir(free_dir))  # 「dataset/free」ディレクトリ内のファイル数をカウントして、free_countウィジェットの値を更新します。

########################################
# 「blocked」用に画像を保存します。
# また、「blocked」として保存されているファイル数の値を更新します。
########################################
def save_blocked():
    global blocked_dir, blocked_count  # blocked_dirとblocked_countの定義とsave_blocked()関数の実行はともにメインスレッド上で行われるため、このglobal定義は省略可能です。
    save_snapshot(blocked_dir)  # save_snapshot()関数を呼び出して、「dataset/blocked」ディレクトリに現在imageウィジェットが持つjpeg画像データをjpgファイルとして保存します。
    blocked_count.value = len(os.listdir(blocked_dir))  # 「dataset/blocked」ディレクトリ内のファイル数をカウントして、blocked_countウィジェットの値を更新します。

########################################
# ボタンがクリックされたときに実行するコールバックを登録します。
# ここではボタンがクリックされた時に呼び出されるon_click()関数で、
# 画像保存の関数を呼び出すように定義します。
# on_click()関数は呼び出す関数に自身（ボタンウィジェットインスタンス）を引数として提供しますが、
# 画像保存関数ではボタン自身の情報は不要です。
# そこで、lambda関数を使ってボタンウィジェットインスタンスをパラメータ x として受けたうえで、
# それを無視するように実装します。
########################################
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

> これでボタンをクリックすると画像が保存されるようになりました。

### 画面を表示して始めよう
次のコードを実行すると、画面とボタンが表示されます。
1. 旋回したい場所にJetBotを置き「blockedに保存」を押します。
2. 直進したい場所にJetBotを置き「freeに保存」を押します。
3. 1, 2の作業を繰り返します。

ラベル付けのためにいくつかのTipsがあります。

1. さまざまな場所、角度での試行
2. 異なる照明、環境での試行
3. さまざまな壁、棚、人の足、ダンボールなどの障害物での試行
4. 異なる床の模様やパターン、なめらかさや、ガラスなどでの試行
5. データ件数はかたよりすぎないようにバランスをとる

> 最終的に、JetBotが現実の世界で遭遇するシナリオのデータが多いほど、衝突回避の挙動は正確になります。
大量のデータだけでなく、（上記のヒントで説明したような）*さまざまな*データを取得する事が重要です。  
> 学習コードの都合上、各クラスの画像が少なくとも50枚程度は必要になります。（50枚はテスト用のデータに利用されます。）

「free」と「blocked」のどちらか一方にデータがかたより過ぎると、学習時にデータ件数が多い方だけを覚えてしまいます（片方だけを予測した方が精度がよくなってしまうため）。

In [ ]:
########################################
# これまでに作成してきたウィジェットを表示します。
########################################
display(image)
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

### カメラ停止

最後に、他のノートブックでカメラを使うために、このノートブックで使ったカメラを停止しておきます。

In [ ]:
########################################
# カメラを停止します。
########################################
camera.stop()

## 次
十分なデータを収集したら、トレーニングに進みます。  
JetBot本体で学習する場合は、このノートブックを閉じてからJupyter左側にある「Running Terminals and Kernels」を選択して「02_data_collection_JP.ipynb」の横にある「SHUT DOWN」をクリックしてJupyter Kernelをシャットダウンしてから[02_train_model_JP.ipynb](02_train_model_JP.ipynb)に進んでください。  